# Download the weights and model config

In [ ]:
!rm -rf *
!wget -q -O "video.avi" "https://drive.switch.ch/index.php/s/3b3bdbd6f8fb61e05d8b0560667ea992/download?path=%2Fvideos%2Fplanes&files=Video_9.avi" 
!wget -q -O "yolov3.weights" "https://pjreddie.com/media/files/yolov3.weights"
!wget -q -O "yolov3.cfg" "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
!wget -q -O "coco.name" "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

# Imports

In [ ]:
import cv2
import time
import numpy as np
import seaborn as sns
import os
from pprint import pprint
from IPython.display import Video

In [ ]:
def load_input_image(image_path):
    test_img = cv2.imread(image_path)
    height, width, depth = test_img.shape
    return test_img, height, width

def yolov3(weights, config, coco_names):
    net = cv2.dnn.readNetFromDarknet(config, weights)
    classes = open(coco_names).read().strip().split("\n")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, classes, output_layers

In [ ]:
def perform_detection(net, img, output_layers, w, h, confidence_threshold):
    blob = cv2.dnn.blobFromImage(img, 1 / 255., (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > confidence_threshold:
                center_x, center_y, width, height = list(map(int, detection[0:4] * [w, h, w, h]))
                top_left_x = int(center_x - (width / 2))
                top_left_y = int(center_y - (height / 2))
                boxes.append([top_left_x, top_left_y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids


def draw_boxes_labels(boxes, confidences, class_ids, classes, img, colors, confidence_threshold, NMS_threshold, disp_scores):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, NMS_threshold)
    FONT = cv2.FONT_HERSHEY_SIMPLEX

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[class_ids[i]], confidences[i]) if disp_scores else "{}".format(classes[class_ids[i]])
            (text_w, text_h) = cv2.getTextSize(text, FONT, fontScale=0.6, thickness=2)[0]
            text_offset_x, text_offset_y = 4, -4
            cv2.rectangle(img, (x-1, y-1), (x + text_offset_x + text_w + 4, y + text_offset_y - text_h - 5), color, cv2.FILLED)
            cv2.putText(img, text, (x + text_offset_x, y + text_offset_y - 1), FONT, 0.6, (0, 0, 0), 2, cv2.LINE_AA)

    return img

In [ ]:
def dectection_video_file(video_path, yolo_weights, yolo_cfg, coco_names, confidence_threshold, nms_threshold, disp_scores):
    net, classes, output_layers = yolov3(yolo_weights, yolo_cfg, coco_names)
    colors = ((np.array(sns.color_palette("husl", len(classes))) * 255)).astype(int)

    video = cv2.VideoCapture(video_path)
    out_dir = os.path.join(os.path.dirname(video_path), 'output')
    out_name = os.path.splitext(os.path.basename(video_path))[0] + '.avi'

    frame_width = int(video.get(3))
    frame_height = int(video.get(4))

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    out = cv2.VideoWriter(os.path.join(out_dir, out_name), cv2.VideoWriter_fourcc(*"MJPG"), 20.0, (frame_width, frame_height))
    
    while True:
        ret, image = video.read()
        if ret == False:
            break

        height, width, depth = image.shape
        boxes, confidences, class_ids = perform_detection(net, image, output_layers, width, height, confidence_threshold)
        final_img = draw_boxes_labels(boxes, confidences, class_ids, classes, image, colors, confidence_threshold, nms_threshold, disp_scores)
        out.write(final_img)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    video.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
video_path = "./video.avi"
image_path = None
yolo_weights = "./yolov3.weights"
yolo_cfg     = "./yolov3.cfg"
coco_names   = "./coco.name" 
confidence_threshold = 0.6
nms_threshold = 0.4


dectection_video_file(video_path, yolo_weights, yolo_cfg, coco_names, confidence_threshold, nms_threshold, True)


In [ ]:
!ffmpeg -i output/video.avi -c:a copy -c:v vp9 -b:v 100K out.mp4

In [ ]:
Video("out.mp4")